In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import pandas as pd

In [2]:
# Copiar o comando abaixo da escolha do SUSano
ano = '2022'; CNES = '2602857'; IBGE = '291080'; forma = '030309-A'

In [7]:
dtypes_EMA = {'CNES':str,
             'IBGE_Res':str,
             'IBGE_Estab': str,
             'IBGE_CAPITAL': str,
             'CO_GRUPO': str,
             'CO_GRUPO_SUBGRUPO': str}

In [8]:
df_capitais = pd.read_csv(f'../Dados/{ano}/df_capitais.zip', dtype=dtypes_EMA)
df_capitais['IBGE_UF'] = df_capitais['IBGE_CAPITAL'].str[:2]
ibge_uf_to_nome_uf = df_capitais.set_index(['IBGE_UF'])['NOME_UF']
ibge_uf_to_ibge_capital = df_capitais.set_index(['IBGE_UF'])['IBGE_CAPITAL']

In [9]:
UF = ibge_uf_to_nome_uf[IBGE[:2]]
IBGE_cidade = IBGE
IBGE_capital = ibge_uf_to_ibge_capital[IBGE[:2]]
UF, IBGE_cidade, IBGE_capital

('Bahia', '291080', '292740')

In [10]:
def tabnet_cidade_e_capital(driver, ano, UF, IBGE_cidade, IBGE_capital, forma):
    url_sih = "https://datasus.saude.gov.br/acesso-a-informacao/producao-hospitalar-sih-sus/"
    url_sia = 'https://datasus.saude.gov.br/acesso-a-informacao/producao-ambulatorial-sia-sus/'
    ends_with_sih = 'qr'
    ends_with_sia = 'qb'
    id_botão_forma_sih = 'fig12'
    id_botão_forma_sia = 'fig10'

    if forma.endswith('H'):
        url = url_sih
        ends_with = ends_with_sih
        id_botão_forma = id_botão_forma_sih
    else:
        url = url_sia
        ends_with = ends_with_sia
        id_botão_forma = id_botão_forma_sia

    forma = forma[:6]

    driver.get(url)
    print(f'{driver.title=}')

    radiobuttons = driver.find_elements(By.NAME, "radiobutton")
    for r in radiobuttons:
        if r.get_attribute("value").endswith(ends_with):
            r.click()
    # endswith seleciona Dados Consolidados por local de residência, a partir de 2008

    select_abrangência = Select(driver.find_element(By.ID, 'mySelect'))
    select_abrangência.select_by_visible_text(UF)
    # Tabnet muda para página de seleção

    meses = 'Jan Fev Mar Abr Mai Jun Jul Ago Set Out Nov Dez'.split()
    períodos = [f'{mês}/{ano}' for mês in meses]

    select_período = Select(driver.find_element(By.ID, 'A'))
    for período in períodos:
        select_período.select_by_visible_text(período)
    print('Períodos selecionados:')
    for opc in select_período.all_selected_options:
        if not opc.text in períodos:
            select_período.deselect_by_visible_text(opc.text)
            print(f'{opc.text} eliminado')
        else:
            print(opc.text, end=', ')

    botão_município = driver.find_element(By.ID, 'fig1')
    botão_município.click()  # Abre a seleção de municípios
    select_município = Select(driver.find_element(By.NAME, 'SMunicípio'))
    opções_município = select_município.options
    for opc in opções_município:
        for ibge in [IBGE_cidade, IBGE_capital]:
            if opc.text.startswith(ibge):
                select_município.select_by_visible_text(opc.text)
    select_município.deselect_by_visible_text('Todas as categorias')
    print('Municípios selecionados:')
    for opc in select_município.all_selected_options:
        print(opc.text)
    botão_município.click()  # Fecha a seleção de municípios

    botão_forma = driver.find_element(By.ID, id_botão_forma)
    botão_forma.click()  # Abre a seleção de formas de organização
    select_forma = Select(driver.find_element(By.NAME, 'SForma_organização'))
    opções_forma = select_forma.options
    for opc in opções_forma:
        if opc.text.startswith(forma):
            select_forma.select_by_visible_text(opc.text)
    select_forma.deselect_by_visible_text('Todas as categorias')
    print('Formas de organização selecionadas:')
    for opc in select_forma.all_selected_options:
        print(opc.text)
    botão_forma.click()  # Fecha a seleção de formas de organização

    botão_mostra = driver.find_element(By.NAME, 'mostre')
    botão_mostra.click()
    return 'ok'

In [11]:
def tabnet_distribuição(driver, ano, UF, IBGE_cidade, forma):
    url_sih = "https://datasus.saude.gov.br/acesso-a-informacao/producao-hospitalar-sih-sus/"
    url_sia = 'https://datasus.saude.gov.br/acesso-a-informacao/producao-ambulatorial-sia-sus/'
    ends_with_sih = 'qr'
    ends_with_sia = 'qb'
    id_botão_forma_sih = 'fig12'
    id_botão_forma_sia = 'fig10'
    
    
    if forma.endswith('H'):
        url = url_sih
        ends_with = ends_with_sih
        id_botão_forma = id_botão_forma_sih
    else:
        print(f'O Tabnet SIA não mostra a distribuição por estabelecimento')
        return 'Erro'
        
    forma = forma[:6]
    
    driver.get(url)
    print(f'{driver.title=}')
    
    radiobuttons = driver.find_elements(By.NAME, "radiobutton")
    for r in radiobuttons:
        if r.get_attribute("value").endswith(ends_with):
            r.click()
    # endswith seleciona Dados Consolidados por local de residência, a partir de 2008
    
    select_abrangência = Select(driver.find_element(By.ID, 'mySelect'))
    select_abrangência.select_by_visible_text(UF)
    # Tabnet muda para página de seleção
    
    meses = 'Jan Fev Mar Abr Mai Jun Jul Ago Set Out Nov Dez'.split()
    períodos = [f'{mês}/{ano}' for mês in meses]
    
    select_período = Select(driver.find_element(By.ID, 'A'))
    for período in períodos:
        select_período.select_by_visible_text(período)
    print('Períodos selecionados:')
    for opc in select_período.all_selected_options:
        if not opc.text in períodos:
            select_período.deselect_by_visible_text(opc.text)
            print(f'{opc.text} eliminado')
        else:
            print(opc.text, end=', ')
    print('\n')
        
    botão_município = driver.find_element(By.ID, 'fig1')
    botão_município.click() # Abre a seleção de municípios   
    select_município = Select(driver.find_element(By.NAME, 'SMunicípio'))
    opções_município = select_município.options
    for opc in opções_município:
        if opc.text.startswith(IBGE_cidade):
            select_município.select_by_visible_text(opc.text)
    select_município.deselect_by_visible_text('Todas as categorias')
    print('Municípios selecionados:')
    for opc in select_município.all_selected_options:
        print(opc.text)
    botão_município.click() # Fecha a seleção de municípios
    
    botão_forma = driver.find_element(By.ID, id_botão_forma)
    botão_forma.click() # Abre a seleção de formas de organização
    select_forma = Select(driver.find_element(By.NAME, 'SForma_organização'))
    opções_forma = select_forma.options
    for opc in opções_forma:
        if opc.text.startswith(forma):
            select_forma.select_by_visible_text(opc.text)
    select_forma.deselect_by_visible_text('Todas as categorias')
    print('Formas de organização selecionadas:')
    for opc in select_forma.all_selected_options:
        print(opc.text)
    botão_forma.click() # Fecha a seleção de formas de organização
    
    select_linha = Select(driver.find_element(By.NAME, 'Linha'))
    select_linha.select_by_visible_text('Estabelecimento')
    
    check_ordena = driver.find_element(By.ID, "O")
    check_ordena.click()
    
    botão_mostra = driver.find_element(By.NAME, 'mostre')
    botão_mostra.click()
    return 'ok'

In [12]:
driver = webdriver.Chrome() 

In [13]:
tabnet_cidade_e_capital(driver, ano, UF, IBGE_cidade, IBGE_capital, forma)

driver.title='Produção Ambulatorial (SIA/SUS) – DATASUS'
Períodos selecionados:
Out/2023 eliminado
Dez/2022, Nov/2022, Out/2022, Set/2022, Ago/2022, Jul/2022, Jun/2022, Mai/2022, Abr/2022, Mar/2022, Fev/2022, Jan/2022, Municípios selecionados:
291080 FEIRA DE SANTANA
292740 SALVADOR
Formas de organização selecionadas:
030309 Tratamento de doenças do sistema osteomuscular e do tecido conjuntivo


'ok'

In [ ]:
tabnet_distribuição(driver, ano, UF, IBGE_cidade, forma)

In [ ]:
driver.close()

In [ ]:
driver = webdriver.Chrome()

In [ ]:
def ficha_cnes(driver, CNES):
    url_cnes = 'http://cnes.datasus.gov.br/pages/estabelecimentos/consulta.jsp'

    driver.get(url_cnes)

    input_pesquisa = driver.find_element(By.ID, 'pesquisaValue')
    input_pesquisa.send_keys(CNES)

    botão_pesquisa = driver.find_element(By.XPATH, '//button[text()="Pesquisar"]')
    botão_pesquisa.click()


In [ ]:
link_ficha = ficha_cnes(driver, CNES)

In [ ]:
link_ficha.get_attribute("href")

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
ActionChains(driver).move_to_element(link_ficha).click(link_ficha).perform()